<a href="https://colab.research.google.com/github/MTS0222/Cafe-sales-SQL/blob/main/Cafe_sales_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/datasets/ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training

https://drive.google.com/file/d/1Z9h1ZlU_b8QYWuP16xGFxiEhRvYtUIm-/view?usp=drive_link

In [6]:
!pip install gdown

!gdown --id 1Z9h1ZlU_b8QYWuP16xGFxiEhRvYtUIm-

!unzip dirty_cafe_sales.zip -d /content/

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Z9h1ZlU_b8QYWuP16xGFxiEhRvYtUIm-
To: /content/dirty_cafe_sales.zip
100% 108k/108k [00:00<00:00, 85.8MB/s]
Archive:  dirty_cafe_sales.zip
  inflating: /content/dirty_cafe_sales.csv  


In [7]:
import pandas as pd

df = pd.read_csv('/content/dirty_cafe_sales.csv')

In [3]:
import sqlite3

conn = sqlite3.connect(':memory:')

In [ ]:
#df = pd.read_csv('dirty_cafe_sales.csv')

In [8]:
df.columns = df.columns.str.replace(' ', '_')

In [9]:
df.to_sql('cafe_sql', conn, if_exists='replace', index=False)

10000

In [ ]:
sql = '''select * from cafe_sql t limit 20'''

In [ ]:
def select(sql):
  return pd.read_sql(sql, conn)

In [ ]:
#DataFrame with menu table (additional material from kaggle)
menu_df = pd.DataFrame({
    'item': [
        'Espresso','Coffee','Tea','Sandwich',
        'Salad','Cake','Cookie','Smoothie','Juice'
    ],
    'Correct_price': [1.5,2,1.5,4,5,3,1,4,3]
})

menu_df.to_sql('menu', conn, if_exists='replace', index=False)

9

In [ ]:
select(sql)

,Transaction_ID,Item,Quantity,Price_Per_Unit,Total_Spent,Payment_Method,Location,Transaction_Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
5,TXN_2602893,Smoothie,5,4.0,20.0,Credit Card,None,2023-03-31
6,TXN_4433211,UNKNOWN,3,3.0,9.0,ERROR,Takeaway,2023-10-06
7,TXN_6699534,Sandwich,4,4.0,16.0,Cash,UNKNOWN,2023-10-28
8,TXN_4717867,None,5,3.0,15.0,None,Takeaway,2023-07-28
9,TXN_2064365,Sandwich,5,4.0,20.0,None,In-store,2023-12-31


# **CLEANING**

In this chunk of code, i check whether TXN corresponds to a transaction or if there are different prefixes.

In [ ]:
sql = '''
SELECT DISTINCT Substr(Transaction_ID, 1, 3) AS short_id
FROM cafe_sql
'''

select(sql)

,short_id
0,TXN


I check the unique values for my short(not so many unique values) categorical variables to see if there are any misspellings, additional spaces, or other imperfections.

In [ ]:
sql = '''
SELECT DISTINCT Item
FROM cafe_sql;
'''

select(sql)

,Item
0,Coffee
1,Cake
2,Cookie
3,Salad
4,Smoothie
5,UNKNOWN
6,Sandwich
7,None
8,ERROR
9,Juice


In [ ]:
sql = '''
SELECT DISTINCT Payment_Method
FROM cafe_sql;
'''

select(sql)

,Payment_Method
0,Credit Card
1,Cash
2,UNKNOWN
3,Digital Wallet
4,ERROR
5,None


In [ ]:
sql = '''
SELECT DISTINCT Location
FROM cafe_sql;
'''

select(sql)

,Location
0,Takeaway
1,In-store
2,UNKNOWN
3,None
4,ERROR


I found that there were many different categories used for missing values.
To make the code shorter and avoid writing the same logic for each column, i decided to create a Python function.
I replaced all missing values with 'Unknown', to decide later whether to leave them, convert them to NaN, or impute them.

In [ ]:
def fix_unknown(val):
    bad_vals = ['ERROR', 'None', 'UNKNOWN']
    if val is None:
        return 'Unknown'
    if isinstance(val, str) and val in bad_vals:
        return 'Unknown'
    return val

conn.create_function("FIX", 1, fix_unknown)

I created a new table with data where all missing values are made uniform.

In [ ]:
sql = '''
CREATE TABLE cafe_unknown AS
SELECT
  Transaction_ID AS id,
  FIX(Item) AS item,
  FIX(Quantity) AS quantity,
  FIX(Price_Per_Unit) AS unit_price,
  FIX(Total_Spent) AS total_price,
  FIX(Payment_Method) AS payment_method,
  FIX(Location) AS location,
  FIX(Transaction_Date) AS date
FROM cafe_sql;
'''
conn.execute(sql)
conn.commit()

In [ ]:
select('SELECT * FROM cafe_unknown LIMIT 10')

,id,item,quantity,unit_price,total_price,payment_method,location,date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,Unknown,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,Unknown,Unknown,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
5,TXN_2602893,Smoothie,5,4.0,20.0,Credit Card,Unknown,2023-03-31
6,TXN_4433211,Unknown,3,3.0,9.0,Unknown,Takeaway,2023-10-06
7,TXN_6699534,Sandwich,4,4.0,16.0,Cash,Unknown,2023-10-28
8,TXN_4717867,Unknown,5,3.0,15.0,Unknown,Takeaway,2023-07-28
9,TXN_2064365,Sandwich,5,4.0,20.0,Unknown,In-store,2023-12-31


Since the unknown values are now standardized, I created another table where I joined the additional menu table (containing correct item prices).
This allowed me to check if the unit_price values were corrupted or missing, and to impute them if the item name was known. I use left join because i want to keep all columns from my cafe table and just add one column from menu.

In [ ]:
sql = '''
CREATE TABLE cafe_unknown_new AS
SELECT c.*, m.Correct_price
FROM cafe_unknown AS c
LEFT JOIN menu as m
  ON c.item = m.item
'''
conn.execute(sql)
conn.commit()

In this step, I impute the 'Unknown' values using known columns.
I use CASE to create new columns with corrected or imputed values based on available information.
I use CAST() to convert the values into real (numeric) format, because originally some of the columns (like unit_price or quantity) were stored as text strings due to errors or missing entries.

In [ ]:
sql = '''
CREATE TABLE cafe_clean AS
SELECT
  id,
  item,
  payment_method,
  location,
  date,
  Correct_price,

  CASE
    WHEN unit_price = 'Unknown' THEN CAST(Correct_price AS REAL)
    ELSE unit_price
  END AS unit_price,

  CASE
    WHEN total_price = 'Unknown'
         AND quantity != 'Unknown'
         AND unit_price != 'Unknown'
      THEN CAST(quantity * unit_price AS REAL)
    ELSE total_price
  END AS total_price,

  CASE
    WHEN quantity = 'Unknown'
         AND total_price != 'Unknown'
         AND unit_price != 'Unknown'
     THEN CAST(total_price / unit_price AS REAL)
    ELSE quantity
  END AS quantity

FROM cafe_unknown_new;
'''
conn.execute(sql)
conn.commit()

In [ ]:
select('SELECT * FROM cafe_clean LIMIT 10')

,id,item,payment_method,location,date,Correct_price,unit_price,total_price,quantity
0,TXN_1961373,Coffee,Credit Card,Takeaway,2023-09-08,2.0,2.0,4.0,2
1,TXN_4977031,Cake,Cash,In-store,2023-05-16,3.0,3.0,12.0,4
2,TXN_4271903,Cookie,Credit Card,In-store,2023-07-19,1.0,1.0,4.0,4
3,TXN_7034554,Salad,Unknown,Unknown,2023-04-27,5.0,5.0,10.0,2
4,TXN_3160411,Coffee,Digital Wallet,In-store,2023-06-11,2.0,2.0,4.0,2
5,TXN_2602893,Smoothie,Credit Card,Unknown,2023-03-31,4.0,4.0,20.0,5
6,TXN_4433211,Unknown,Unknown,Takeaway,2023-10-06,NaN,3.0,9.0,3
7,TXN_6699534,Sandwich,Cash,Unknown,2023-10-28,4.0,4.0,16.0,4
8,TXN_4717867,Unknown,Unknown,Takeaway,2023-07-28,NaN,3.0,15.0,5
9,TXN_2064365,Sandwich,Unknown,In-store,2023-12-31,4.0,4.0,20.0,5


I checked whether there was a mismatch between the prices in the menu and the prices in my cafe table, and there were none.

In [ ]:
sql = '''
SELECT
  correct_price,
  unit_price
FROM cafe_clean
WHERE
  unit_price != correct_price
  AND unit_price != 'Unknown'
  AND correct_price IS NOT NULL;
'''
select(sql)

,Correct_price,unit_price


I want to find all rows that have more than three unknown values, as at this point we would not be able to extract any meaningful information from them (there are 11 such rows). I use CASE to flag fields that are 'Unknown', sum those flags, and then filter rows where the sum is greater than 3.

In [ ]:
sql = '''
SELECT
  id,
  item,
  payment_method,
  location,
  date,
  unit_price,
  quantity,
  total_price
FROM cafe_clean
WHERE
  (CASE WHEN item = 'Unknown' THEN 1 ELSE 0 END) +
  (CASE WHEN payment_method = 'Unknown' THEN 1 ELSE 0 END) +
  (CASE WHEN location = 'Unknown' THEN 1 ELSE 0 END) +
  (CASE WHEN unit_price = 'Unknown' THEN 1 ELSE 0 END) +
  (CASE WHEN total_price = 'Unknown' THEN 1 ELSE 0 END) +
  (CASE WHEN quantity = 'Unknown' THEN 1 ELSE 0 END) +
  (CASE WHEN id = 'Unknown' THEN 1 ELSE 0 END) +
  (CASE WHEN date = 'Unknown' THEN 1 ELSE 0 END)
> 3;
'''
select(sql)

,id,item,payment_method,location,date,unit_price,quantity,total_price
0,TXN_2962976,Juice,Unknown,Unknown,2023-03-17,3.0,Unknown,Unknown
1,TXN_1185118,Unknown,Unknown,Unknown,Unknown,3.0,5,15.0
2,TXN_7524977,Unknown,Unknown,Unknown,2023-12-09,None,4,Unknown
3,TXN_6297232,Coffee,Unknown,Unknown,2023-04-07,2.0,Unknown,Unknown
4,TXN_6500126,Smoothie,Unknown,Unknown,Unknown,4.0,2,Unknown
5,TXN_9455737,Unknown,Unknown,Unknown,Unknown,5.0,3,15.0
6,TXN_6184247,Unknown,Unknown,Unknown,Unknown,3.0,1,3.0
7,TXN_6859249,Cookie,Unknown,Unknown,Unknown,1.0,Unknown,2.0
8,TXN_2194660,Unknown,Unknown,Unknown,Unknown,5.0,1,5.0
9,TXN_9669616,Coffee,Unknown,Unknown,2023-06-03,2.0,Unknown,Unknown


After checking that there are only 11 such rows, I dropped them.

In [ ]:
sql = '''
DELETE FROM cafe_clean
WHERE
  (CASE WHEN item = 'Unknown' THEN 1 ELSE 0 END) +
  (CASE WHEN payment_method = 'Unknown' THEN 1 ELSE 0 END) +
  (CASE WHEN location = 'Unknown' THEN 1 ELSE 0 END) +
  (CASE WHEN unit_price = 'Unknown' THEN 1 ELSE 0 END) +
  (CASE WHEN total_price = 'Unknown' THEN 1 ELSE 0 END) +
  (CASE WHEN quantity = 'Unknown' THEN 1 ELSE 0 END) +
  (CASE WHEN id = 'Unknown' THEN 1 ELSE 0 END) +
  (CASE WHEN date = 'Unknown' THEN 1 ELSE 0 END)
> 3;
'''
conn.execute(sql)
conn.commit()

I checked for duplicates (assuming that they would have the same id). I grouped the data by id and count(*) the number of rows for each id. If the count is greater than one, i call it a duplicate. But there were none.


In [ ]:
sql = '''
SELECT
  id,
  COUNT(*) AS count_id
FROM cafe_clean
GROUP BY id
HAVING COUNT(*) > 1;
'''
select(sql)

,id,count_id


# **EXPLORATION**



---


 1. Payments method used on each location with revenue summarised, and average spend per payment method
 2. Number of transactions per month
 3. Best selling items
 4. Best selling items by location
 5. Top 3 items sold each month
 6. Busiest day of the week


---



**1. Payment methods used on each location with revenue summarised, and average spend per payment method**

Breakdown : i grouped data by location and payment method using GROUP BY, counted transactions (not quantity sold) thats why we use COUNT(*), calculated averages AVG() and totals SUM() and ranked payment methods inside each location with window function (ROW_NUMBER).

In [ ]:
sql = '''
SELECT
  ROW_NUMBER() OVER(PARTITION BY location ORDER BY COUNT(*) DESC) AS row_num,
  location,
  payment_method,
  COUNT(*) AS transaction_count,
  AVG(total_price) AS average_transaction_value,
  SUM(total_price) AS total_spent
FROM cafe_clean
  WHERE
  payment_method != 'Unknown' AND location != 'Unknown'
GROUP BY location, payment_method
ORDER BY location, transaction_count DESC;
'''
select(sql)

,row_num,location,payment_method,transaction_count,average_transaction_value,total_spent
0,1,In-store,Cash,702,9.088319,6380.0
1,2,In-store,Digital Wallet,695,9.428058,6552.5
2,3,In-store,Credit Card,681,8.784141,5982.0
3,1,Takeaway,Digital Wallet,744,8.421371,6265.5
4,2,Takeaway,Credit Card,669,8.832586,5909.0
5,3,Takeaway,Cash,664,8.859940,5883.0


**2. Number of transactions per month**

I grouped dates into months and counted rows.

In [ ]:
sql = '''
SELECT
  strftime('%Y-%m', date) AS month,
  COUNT(*) AS transactions
FROM cafe_clean
GROUP BY month
ORDER BY month;
'''
select(sql)

,month,transactions
0,None,452
1,2023-01,818
2,2023-02,727
3,2023-03,826
4,2023-04,773
5,2023-05,777
6,2023-06,817
7,2023-07,791
8,2023-08,803
9,2023-09,788


**3. Best selling items (overall)**
I add up how many units of each item were sold with SUM(), group all sales by item name, order the items from the most sold to the least sold with DESC in ORDER BY.

In [ ]:
sql = '''
SELECT
  item,
  SUM(quantity) AS total_quantity_sold
FROM cafe_clean
GROUP BY item
ORDER BY total_quantity_sold DESC;
'''
select(sql)

,item,total_quantity_sold
0,Coffee,3534.0
1,Juice,3505.0
2,Salad,3468.0
3,Cake,3462.0
4,Sandwich,3424.0
5,Smoothie,3328.0
6,Tea,3292.0
7,Cookie,3228.0
8,Unknown,2882.0


**4. Best selling items by location**

Breakdown: i grouped data by location and item using GROUP BY, added up total quantity sold using SUM(), summarized total revenue for each item SUM(), ranked items inside each location by quantity sold with ROW_NUMBER() window function so we have separate ranking for each location (although it doesnt look as beautiful with the default python index on the left) and ordered by location and total_quantity.

In [ ]:
sql = '''
SELECT
  ROW_NUMBER() OVER(PARTITION BY location ORDER BY SUM(quantity) DESC) AS row_num,
  location,
  item,
  SUM(quantity) AS total_quantity_sold,
  SUM(total_price) AS total_revenue
FROM cafe_clean
WHERE
  location != 'Unknown'
  AND item != 'Unknown'
GROUP BY location, item
ORDER BY location, total_quantity_sold DESC;
'''
select(sql)

,row_num,location,item,total_quantity_sold,total_revenue
0,1,In-store,Salad,1132.0,5640.0
1,2,In-store,Sandwich,1127.0,4496.0
2,3,In-store,Juice,1103.0,3309.0
3,4,In-store,Tea,1012.0,1515.0
4,5,In-store,Cake,986.0,2973.0
5,6,In-store,Cookie,972.0,965.0
6,7,In-store,Coffee,970.0,1940.0
7,8,In-store,Smoothie,953.0,3812.0
8,1,Takeaway,Coffee,1091.0,2182.0
9,2,Takeaway,Cookie,1090.0,1088.0


**5. Top 3 items sold each month**

Breakdown: i extracted month using strftime('%Y-%m', date), grouped by month and item, summed total quantity, used ROW_NUMBER() window function partitioned by month to rank the items and selected only top 3 items for each month (row_num <= 3).

In [ ]:
sql = '''
WITH monthly_items AS (
  SELECT
    strftime('%Y-%m', date) AS month,
    item,
    SUM(quantity) AS total_quantity_sold,
    ROW_NUMBER() OVER (PARTITION BY strftime('%Y-%m', date) ORDER BY SUM(quantity) DESC) AS row_num
  FROM cafe_clean
  WHERE item != 'Unknown' AND month != 'Unknown'
  GROUP BY month, item
)

SELECT
  month,
  item,
  total_quantity_sold
FROM monthly_items
WHERE row_num <= 3
ORDER BY month, total_quantity_sold DESC;
'''
select(sql)

,month,item,total_quantity_sold
0,2023-01,Sandwich,341.0
1,2023-01,Salad,315.0
2,2023-01,Juice,306.0
3,2023-02,Coffee,286.0
4,2023-02,Juice,282.0
5,2023-02,Tea,262.0
6,2023-03,Coffee,325.0
7,2023-03,Juice,308.0
8,2023-03,Sandwich,294.0
9,2023-04,Juice,305.0


**6. Busiest day of the week**

Breakdown: i used strftime('%w', date) to get code of day of the week 0-6, i used CASE to map numbers into day names, grouped transactions by weekday name, calculated average quantity sold per day and ordered the results to find the day with the highest average sales. i used LIMIT 1, but then i changed my mind and decided to leave them all (so i removed LIMIT).

In [ ]:
sql = '''
SELECT
  CASE strftime('%w', date)
    WHEN '0' THEN 'Sunday'
    WHEN '1' THEN 'Monday'
    WHEN '2' THEN 'Tuesday'
    WHEN '3' THEN 'Wednesday'
    WHEN '4' THEN 'Thursday'
    WHEN '5' THEN 'Friday'
    WHEN '6' THEN 'Saturday'
  END AS week_day,
  AVG(quantity) AS average_sales
FROM cafe_clean
  WHERE
  week_day != 'Unknown'
GROUP BY week_day
ORDER BY average_sales DESC;
'''
select(sql)

,week_day,average_sales
0,Tuesday,3.101449
1,Thursday,3.031884
2,Friday,3.025974
3,Sunday,3.008696
4,Saturday,2.987463
5,Monday,2.981910
6,Wednesday,2.944817
